![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Disponibilización de modelos

En este notebook aprenderá a guardar un modelo y a disponibilizarlo como una API con la librería Flask. Una API (interfaz de programación de aplicaciones) es un conjunto de definiciones y protocolos que permiten que servicios, en este caso modelos, retornen resultados y respuestas sin necesidad de saber cómo están implementados.

## Instrucciones Generales:

Este notebook esta compuesto por dos secciones. En la primera sección, usted beberá entrenar y guardar (exportar) un modelo de random forest para predecir si una URL es phishing (fraudulenta) o no. En la segunda parte, usará el modelo entrenado y lo disponibilizara usando la libreria *Flask*. En el siguente paper puede conocer más detalles de la base de datos que usaremos y del problema: *A. Correa Bahnsen, E. C. Bohorquez, S. Villegas, J. Vargas, and F. A. Gonzalez, “Classifying phishing urls using recurrent neural networks,” in Electronic Crime Research (eCrime), 2017 APWG Symposium on. IEEE, 2017, pp. 1–8*. https://albahnsen.files.wordpress.com/2018/05/classifying-phishing-urls-using-recurrent-neural-networks_cameraready.pdf

**Nota:** Para desarrollar la actividad, **descargue el notebook o ábralo desde el repositorio clonado de GitHub del curso** y siga las indicaciones que encontrará en cada una de las celdas de **manera local**. Este laboratorio no se puede ejecutar en la plataforma de Coursera pues requiere exportar e importar archivos relacionados en el repositorio del curso.

## Importar base de datos y librerías

In [1]:
# instalar flask_restplus
!pip install pandas
!pip install numpy
!pip install xgboost
!pip install sklearn
!pip install Flask
!pip install flask_restx
!pip install joblib
#!pip install werkzeug

exportar importar modelo xgboost

https://mljar.com/blog/xgboost-save-load-python/

In [70]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from joblib import load

def predict_genres(Movie):
    
    # Carga modelo
    modelo_ = load('LogReg_pipeline.joblib')
    
     
    # Make prediction
    genres_name = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
           'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical',
           'Mystery', 'News', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western']
    genres_description = []
    genres = ((modelo_.predict_proba([Movie]) >= 0.5)*1)[0]
    if sum(genres) > 0:
        for i in range(len(genres_name)):
            if genres[i] == 1:
                genres_description.append(genres_name[i])
    else:
        genres_description = ['No matches in Generes']
    
    return genres_description


In [72]:
predict_genres('exploration violent society depictions violence reflect shape culture middle class family submits physically mentally torture violence death foisted upon two young unexpected white gloved visitors vacation retreat near lake')

['No matches in Generes']

In [ ]:
#!/usr/bin/python
from flask import Flask
from flask_restx import Api, Resource, fields
#import joblib
#from S3P1_model_deployment import predict_price

app = Flask(__name__)

api = Api(
    app, 
    version='1.0', 
    title='Movie Generes Prediction API',
    description='Movie Generes Prediction API')

ns = api.namespace('predict', 
     description='Movie Generes Classifier')
   
parser = api.parser()

parser.add_argument(
    'Movie', 
    type=str, 
    required=True, 
    help='Movie to be analyzed', 
    location='args')

resource_fields = api.model('Resource', {
    'Generes': fields.String,
})

@ns.route('/')
class GenresApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def post(self):
        args = parser.parse_args()
        
        return {
         "Generes": predict_genres(args['Movie'])
        }, 200
    
    
#if __name__ == '__main__':
#    app.run()
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://172.24.50.160:5000/ (Press CTRL+C to quit)
172.24.50.160 - - [24/May/2022 19:32:07] "GET / HTTP/1.1" 200 -
172.24.50.160 - - [24/May/2022 19:32:07] "GET /swagger.json HTTP/1.1" 200 -
172.24.50.160 - - [24/May/2022 19:32:24] "POST /predict/?Movie=drama HTTP/1.1" 200 -
172.24.50.160 - - [24/May/2022 19:32:34] "POST /predict/?Movie=action HTTP/1.1" 200 -
